In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Chargement des données
data = pd.read_csv('iot23_18column.csv')
data = data.dropna()

# Séparation des instances bénignes du reste de l'ensemble de données
benign_data = data[data['label'] == 0]  # Supposons que le label 0 représente les instances bénignes
X_benign = benign_data.drop(columns=['label'])

# Standardisation des caractéristiques
scaler = StandardScaler()
X_benign_scaled = scaler.fit_transform(X_benign)

# Entraînement du modèle Isolation Forest sur les instances bénignes
isolation_forest = IsolationForest(contamination='auto', random_state=42)  
# Vous devez ajuster ces paramètres en fonction de vos données
isolation_forest.fit(X_benign_scaled)

# Application du modèle Isolation Forest à l'ensemble de données complet
X_all = data.drop(columns=['label'])
X_all_scaled = scaler.transform(X_all)
anomaly_scores = isolation_forest.score_samples(X_all_scaled)

# Extraction des instances prédites comme anomalies (scores négatifs)
predicted_anomalies = data[anomaly_scores < 0]

# Affichage des anomalies prédites
print("Anomalies prédites:", predicted_anomalies)
print("Nombre d'anomalies détectées:", len(predicted_anomalies))

Anomalies prédites:          Unnamed: 0      ts      uid  id.orig_h  id.orig_p  id.resp_h  \
0                 0       2   317928       1360    60905.0     127613   
3                 3       6   961436       1360    34243.0     205149   
8                 8      15   362440       1360    52259.0     455699   
9                 9      17   639458       1360    42207.0       3275   
11               11      20  1377586       1360    40459.0     622407   
...             ...     ...      ...        ...        ...        ...   
1444398       99767  390677  1108747       1362    29930.0     433605   
1444400       99769  388623   764858       1362    29930.0     993050   
1444401       99770  386568  1347417       1362    29930.0     398889   
1444404       99773  388626  1414549       1362    29930.0     590843   
1444405       99774  390678   221139       1362    29930.0    1083421   

         id.resp_p  proto   duration  orig_bytes  resp_bytes  conn_state  \
0             23.0      1  

In [2]:
from sklearn.metrics import confusion_matrix, classification_report

# Convertissons les scores d'anomalies en prédictions binaires (1 pour normal, -1 pour anomalie)
predictions = [1 if score > 0 else -1 for score in anomaly_scores]

# Convertissons les vraies étiquettes (0 pour normal, -1 pour anomalie)
true_labels = [1 if label == 0 else -1 for label in data['label']]

# Calcul de la matrice de confusion et du rapport de classification
conf_matrix = confusion_matrix(true_labels, predictions)
print("Matrice de confusion :\n", conf_matrix)

# Calcul et affichage du rapport de classification
report = classification_report(true_labels, predictions, target_names=['Anomalie', 'Normal'])
print("Rapport de classification :\n", report)


Matrice de confusion :
 [[476734      0]
 [ 67796      0]]


C:\Users\amani\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Rapport de classification :
               precision    recall  f1-score   support

    Anomalie       0.88      1.00      0.93    476734
      Normal       0.00      0.00      0.00     67796

    accuracy                           0.88    544530
   macro avg       0.44      0.50      0.47    544530
weighted avg       0.77      0.88      0.82    544530



C:\Users\amani\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amani\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

# Calcul du score AUC
false_positive_rate, true_positive_rate, thresholds = roc_curve(true_labels, anomaly_scores)
auc_score = roc_auc_score(true_labels, anomaly_scores)

# Trouver le seuil optimal
optimal_idx = np.argmax(true_positive_rate - false_positive_rate)
optimal_threshold = thresholds[optimal_idx]

# Appliquer le seuil optimal pour classer les anomalies
predictions_optimal = [1 if score > optimal_threshold else -1 for score in anomaly_scores]

# Évaluation de la performance avec le nouveau seuil
print(classification_report(true_labels, predictions_optimal,target_names=['Anomalie', 'Normal']))


              precision    recall  f1-score   support

    Anomalie       0.96      0.99      0.98    476734
      Normal       0.91      0.74      0.82     67796

    accuracy                           0.96    544530
   macro avg       0.94      0.87      0.90    544530
weighted avg       0.96      0.96      0.96    544530

